In [1]:
import pandas as pd
import numpy as np
import os, random, warnings, gc, psutil, datetime
from tqdm import tqdm_notebook, tqdm

from multiprocessing import Pool

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

import lightgbm as lgbm

from glob import glob
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import MiniBatchKMeans, KMeans

# Set options
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)
pd.options.display.max_colwidth = 300

warnings.filterwarnings('ignore')

%matplotlib inline
sns.set_palette('bright')

In [10]:
path = '../raw_dataset/'

## Load Dataset (giba code)

In [3]:
df_bus = pd.read_csv(path + 'bus_bts.csv')
df_bus['geton_date'] = pd.to_datetime(df_bus['geton_date'])

# 주중에 정기적으로 타는 사람들
df_bus['weekday'] = df_bus['geton_date'].dt.weekday

df_weekday = df_bus[df_bus['weekday']<5]

df_commuter = df_weekday.groupby(['user_card_id','geton_station_code']).size().reset_index()
df_commuter.columns = ['user_card_id','geton_station_code','num_usage']

df_commuter = df_commuter[df_commuter['num_usage']>=10].reset_index(drop=True)
df_commuter = df_commuter.groupby('geton_station_code')['user_card_id'].count()

df_commuter = df_commuter.reset_index()
df_commuter.columns = ['station_code','regular_commuter_count']

# 해당 정류장에 12시 이후에 몇명이 내렸는지
df_afternoon = df_bus[df_bus['getoff_time']>='12:00:00'][['bus_route_id','getoff_date','getoff_station_code','getoff_time','user_category','user_count']]
df_afternoon_getoff_amount = df_afternoon.groupby(['bus_route_id','getoff_date','getoff_station_code'])['user_count'].sum().reset_index()
df_afternoon_getoff_amount = df_afternoon_getoff_amount.rename(columns = {'user_count' : 'afternoon_takeoff'})

# 해당 정류장에 같은 노선의 버스가 직전 몇분 전에 왔었는지
first_passenger_tagtime = df_bus.groupby(['geton_date', 'bus_route_id', 'vhc_id','geton_station_code'])['geton_time'].min().reset_index()


first_passenger_tagtime = first_passenger_tagtime.sort_values(by=['geton_date','bus_route_id','geton_station_code','geton_time']).reset_index(drop=True)

first_passenger_tagtime['geton_time_second']= first_passenger_tagtime['geton_time'].apply(lambda x: 60*60 *int(x.split(':')[0] ) +\
                                                                                                    60 * int(x.split(':')[1]) +\
                                                                                                          int(x.split(':')[2]) )

first_passenger_tagtime['next_bus_time_diff'] = first_passenger_tagtime.groupby(['geton_date','bus_route_id','geton_station_code'])['geton_time_second'].diff()
date_route_stataion_waittime = first_passenger_tagtime.groupby(['geton_date','bus_route_id','geton_station_code'])['next_bus_time_diff'].mean().reset_index()

date_route_stataion_waittime = date_route_stataion_waittime.groupby(['geton_date','bus_route_id'])['next_bus_time_diff'].mean()
date_route_stataion_waittime =date_route_stataion_waittime.reset_index()

# 6~9, 9~12시 사이에 각기 다른 집단의 사람들이 몇명 탑승했는지
bus_sample = df_bus[['geton_date','geton_station_code','geton_time','user_category','user_count']].copy()
bus_sample['geton_morning'] = bus_sample['geton_time'].apply(lambda x: int(x.split(':')[0]) <=9 )
bus_passender_cluster_count = bus_sample.groupby(['geton_date','geton_station_code','geton_morning','user_category'])['user_count'].sum().reset_index()

bus_passender_cluster_count_morning = bus_passender_cluster_count[bus_passender_cluster_count['geton_morning']==True]
geton_bus_passender_cluster_count_morning = pd.pivot_table( bus_passender_cluster_count_morning, index = ['geton_date', 'geton_station_code'],
                                columns=['user_category'], values = ['user_count'], aggfunc='sum').reset_index()

geton_bus_passender_cluster_count_morning.columns = ['geton_date', 'geton_station_code']  +\
                            ['getin_user_count1_morning','getin_user_count2_morning','getin_user_count4_morning','getin_user_count6_morning','d1','d2','d3','d4']

geton_bus_passender_cluster_count_morning = geton_bus_passender_cluster_count_morning.drop(['d1','d2','d3','d4'],1)


def calculate_getoff_time(val):
    if val <= 9 :
        return 0
    elif val <= 12:
        return 1
    else:
        return 2
    
# 6~9, 9~12시 사이에 각기 다른 집단의 사람들이 몇명 내렸는지
bus_sample = df_bus[['geton_date','getoff_station_code','getoff_time','user_category','user_count']].copy()
bus_sample = bus_sample[bus_sample['getoff_time'].notnull()]
bus_sample['getoff_hour'] =  bus_sample['getoff_time'].apply(lambda x: int(x.split(':')[0]) )
bus_sample['getoff_hour'] =  bus_sample['getoff_hour'].apply(calculate_getoff_time)

bus_passender_cluster_count = bus_sample.groupby(['geton_date','getoff_station_code','getoff_hour','user_category'])['user_count'].sum().reset_index()

takeoff_bus_passender_cluster_count_noon = bus_passender_cluster_count[bus_passender_cluster_count['getoff_hour']==1]


takeoff_bus_passender_cluster_count_noon = pd.pivot_table( takeoff_bus_passender_cluster_count_noon, index = ['geton_date', 'getoff_station_code'],
                                                                    columns=['user_category'], values = ['user_count'], aggfunc='sum').reset_index()



takeoff_bus_passender_cluster_count_noon.columns = ['geton_date', 'getoff_station_code']  +\
                            ['takeoff_user_count1_noon','takeoff_user_count2_noon','takeoff_user_count4_noon','takeoff_user_count6_noon','d1','d2','d3','d4']

takeoff_bus_passender_cluster_count_noon = takeoff_bus_passender_cluster_count_noon.drop(['d1','d2','d3','d4'],1)


## Main Dataset

In [14]:
path

'../raw_dataset/'

In [12]:
train = pd.read_csv(path + 'train.csv', parse_dates =['date'])
test = pd.read_csv(path + 'test.csv', parse_dates =['date'])

In [15]:
# 여기에 전역변수들 몇개 추가했습니다
n_trn = len(train)
target_col = '18~20_ride'
ride_cols = ['6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride']
ride_go_to_work_cols = ['7~8_ride', '8~9_ride', '9~10_ride']
takeoff_cols = ['6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff']
takeoff_go_to_work_cols = ['7~8_takeoff', '8~9_takeoff', '9~10_takeoff']
SEED = 42
n_splits = 5
TARGET = '18~20_ride'

In [16]:
drop_cols = ['id', 'station_name', 'date', 'preds', 'DT_D']

In [17]:
# Make a whole dataset
combined = train.append(test, ignore_index=True)

In [18]:
combined.head()

,10~11_ride,10~11_takeoff,11~12_ride,11~12_takeoff,18~20_ride,6~7_ride,6~7_takeoff,7~8_ride,7~8_takeoff,8~9_ride,8~9_takeoff,9~10_ride,9~10_takeoff,bus_route_id,date,id,in_out,latitude,longitude,station_code,station_name
0,2.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,5.0,0.0,4270000,2019-09-01,0,시외,33.48990,126.49373,344,제주썬호텔
1,5.0,0.0,6.0,0.0,5.0,1.0,0.0,4.0,0.0,4.0,0.0,2.0,0.0,4270000,2019-09-01,1,시외,33.48944,126.48508,357,한라병원
2,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,4270000,2019-09-01,2,시외,33.48181,126.47352,432,정존마을
3,14.0,0.0,16.0,0.0,53.0,0.0,0.0,17.0,0.0,6.0,0.0,26.0,0.0,4270000,2019-09-01,3,시내,33.50577,126.49252,1579,제주국제공항(600번)
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4270000,2019-09-01,4,시내,33.25579,126.41260,1646,중문관광단지입구


## sogna code

In [19]:
df = combined

In [20]:
df['DT_D'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek
df['weekend'] = (df['dayofweek'] >= 5) * 1

In [21]:
df['in_out'] = df['in_out'].map({'시내':0,'시외':1})

In [22]:
df['ride_total'] = df[ride_cols].sum(axis=1)
df['takeoff_total'] = df[takeoff_cols].sum(axis=1)
df['ride_go_to_work'] = df[ride_go_to_work_cols].sum(axis=1)
df['takeoff_go_to_work'] = df[takeoff_go_to_work_cols].sum(axis=1)

In [23]:
import geopy.distance

coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도
df['dis_jejusi'] = [geopy.distance.vincenty((df['latitude'].iloc[i],df['longitude'].iloc[i]), coords_jejusi).km for i in range(len(df))]
df['dis_seoquipo'] = [geopy.distance.vincenty((df['latitude'].iloc[i],df['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(df))]

In [24]:
df['date_bus_route_id_station_code'] = df['date'].astype(str) + '_' + df['bus_route_id'].astype(str) + '_' + df['station_code'].astype(str)
df['date_station_code'] = df['date'].astype(str) + '_' + df['station_code'].astype(str)
df['date_bus_route_id'] = df['date'].astype(str) + '_' + df['bus_route_id'].astype(str)
df['bus_route_id_station_code'] = df['bus_route_id'].astype(str) + '_' + df['station_code'].astype(str)
df['bus_route_id_station_code_weekend'] = df['bus_route_id_station_code'].astype(str) + '_' + df['weekend'].astype(str)

drop_cols += ['date_bus_route_id_station_code', 'date_station_code', 'date_bus_route_id']

In [25]:
df['station_code'] = df['station_code'].astype('category')
df['bus_route_id'] = df['bus_route_id'].astype('category')
df['bus_route_id_station_code'] = df['bus_route_id_station_code'].astype('category')
df['bus_route_id_station_code_weekend'] = df['bus_route_id_station_code_weekend'].astype('category')

In [26]:
def frequency_encoding(data_df, columns, self_encoding=False):
    for col in columns:
        temp_df = data_df[[col]]
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()
        if self_encoding:
            data_df[col] = data_df[col].map(fq_encode)
        else:
            data_df[col+'_fq_enc'] = data_df[col].map(fq_encode)
    return data_df

In [27]:
df = frequency_encoding(df, ['date', 'station_code', 'bus_route_id', 'bus_route_id_station_code', 'date_bus_route_id', 'date_station_code', 'date_bus_route_id_station_code'], self_encoding=False)

In [28]:
def encode_AG(main_columns, groups, aggregations=['mean'], data_df=df, fillna=False, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS
    data_df = data_df.copy()
    for main_column in main_columns:
        for col in groups:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type
                temp_df = data_df[[col,main_column]]
                if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                data_df[new_col_name] = data_df[col].map(temp_df).astype('float32')
                
                if fillna:
                    data_df[new_col_name].fillna(-1,inplace=True)
                
                print("'"+new_col_name+"'",', ',end='')
    
    return data_df

In [29]:
df = encode_AG(ride_go_to_work_cols, ['date', 'date_bus_route_id'], ['mean'], df)

'7~8_ride_date_mean' , '7~8_ride_date_bus_route_id_mean' , '8~9_ride_date_mean' , '8~9_ride_date_bus_route_id_mean' , '9~10_ride_date_mean' , '9~10_ride_date_bus_route_id_mean' , 

## giba code

In [30]:
combined = df.copy()

In [31]:
# Sequence of the station
combined['station_sequence'] = 1
combined['station_reverse_sequence'] = combined[::-1].groupby(['date','bus_route_id'])['station_sequence'].cumsum()[::-1]
combined['station_sequence'] = combined.groupby(['date','bus_route_id'])['station_sequence'].cumsum()

In [32]:
# Change the dtype of "date"
combined['weekday'] = combined['date'].dt.weekday.astype(np.int8)
combined['weekend'] = (combined['weekday'] >= 5) * 1

In [33]:
# Holidays
national_holidays = [datetime.date(2019, 9,12),datetime.date(2019, 9,13), 
                     datetime.date(2019, 9,14),
                     datetime.date(2019, 10,3), datetime.date(2019, 10,9)]

combined['is_national_holiday'] = combined['date'].apply(lambda x: x in national_holidays).astype(np.int8)

In [34]:
# Sum-up the number of passengers for two intervers
morining_getin_cols = ['6~7_ride', '7~8_ride', '8~9_ride']
noon_getin_cols = ['9~10_ride', '10~11_ride', '11~12_ride']

morning_takeoff_cols = ['6~7_takeoff', '7~8_takeoff','8~9_takeoff']
noon_takeoff_cols = ['9~10_takeoff', '10~11_takeoff', '11~12_takeoff']

combined['getin_total'] = combined[morining_getin_cols + noon_getin_cols].sum(axis=1)
combined['takeoff_total'] = combined[morning_takeoff_cols + noon_takeoff_cols].sum(axis=1)

# Morning getin/takeoff & Noon getin/takeoff
combined['morning_getin'] = combined[morining_getin_cols].sum(axis=1)
combined['morning_takeoff'] = combined[morning_takeoff_cols].sum(axis=1)

combined['noon_getin'] = combined[noon_getin_cols].sum(axis=1)
combined['noon_takeoff'] = combined[noon_takeoff_cols].sum(axis=1)

combined = combined.drop(morining_getin_cols  + noon_getin_cols + morning_takeoff_cols + noon_takeoff_cols ,1)

In [35]:
# STATION_CODE

# Sum of passenger per morning (getin)  
station_morning_getin_sum = combined.groupby(['date','station_code'])['morning_getin'].sum().reset_index()
station_morning_getin_sum = station_morning_getin_sum.rename(columns = {'morning_getin': 'station_morning_getin_sum'})

# Sum of passenger per morning (takeoff)  
station_morning_takeoff_sum = combined.groupby(['date','station_code'])['morning_takeoff'].sum().reset_index()
station_morning_takeoff_sum = station_morning_takeoff_sum.rename(columns = {'morning_takeoff': 'station_morning_takeoff_sum'})

# Merge
combined = pd.merge(combined, station_morning_getin_sum , on =['date','station_code'], how='left')
combined = pd.merge(combined, station_morning_takeoff_sum , on =['date','station_code'], how='left')

In [36]:
# BUS_ROUTE

# Sum of passenger per morning (getin)  
bus_route_getin_sum = combined.groupby(['date','bus_route_id'])['morning_getin'].sum().reset_index()
bus_route_getin_sum = bus_route_getin_sum.rename(columns = {'morning_getin': 'bus_route_getin_sum'})

# Sum of passenger per morning (takeoff)  
bus_route_takeoff_sum = combined.groupby(['date','bus_route_id'])['morning_takeoff'].sum().reset_index()
bus_route_takeoff_sum = bus_route_takeoff_sum.rename(columns = {'morning_takeoff': 'bus_route_takeoff_sum'})

# Merge
combined = pd.merge(combined, bus_route_getin_sum , on =['date','bus_route_id'], how='left')
combined = pd.merge(combined, bus_route_takeoff_sum , on =['date','bus_route_id'], how='left')

In [37]:
# STATION_CODE

# mean of passenger per morning (getin)  -- noon getin not working
station_morning_getin_mean = combined.groupby(['date','station_code'])['morning_getin'].mean().reset_index()
station_morning_getin_mean = station_morning_getin_mean.rename(columns = {'morning_getin': 'station_morning_getin_mean'})

# mean of passenger per morning (getin)  -- noon getin not working
station_morning_takeoff_mean = combined.groupby(['date','station_code'])['morning_takeoff'].mean().reset_index()
station_morning_takeoff_mean = station_morning_takeoff_mean.rename(columns = {'morning_takeoff': 'station_morning_takeoff_mean'})

# Merge
combined = pd.merge(combined, station_morning_getin_mean , on =['date','station_code'], how='left')
combined = pd.merge(combined, station_morning_takeoff_mean , on =['date','station_code'], how='left')


In [38]:
# BUS_ROUTE

# mean of passenger per morning (getin)  
bus_route_getin_mean = combined.groupby(['date','bus_route_id'])['morning_getin'].mean().reset_index()
bus_route_getin_mean = bus_route_getin_mean.rename(columns = {'morning_getin': 'bus_route_getin_mean'})

# mean of passenger per morning (takeoff)  
bus_route_takeoff_mean = combined.groupby(['date','bus_route_id'])['morning_takeoff'].mean().reset_index()
bus_route_takeoff_mean = bus_route_takeoff_mean.rename(columns = {'morning_takeoff': 'bus_route_takeoff_mean'})

# Merge
combined = pd.merge(combined, bus_route_getin_mean , on =['date','bus_route_id'], how='left')
combined = pd.merge(combined, bus_route_takeoff_mean , on =['date','bus_route_id'], how='left')

In [39]:
# Kmeans1
df_cluster = combined[['date','bus_route_id','station_code','morning_getin']].copy()
df_cluster['bus_route_id_station_code'] = df_cluster['bus_route_id'].astype(np.str)+'_'+df_cluster['station_code'].astype(np.str)
df_cluster_pivot = pd.pivot_table(data = df_cluster, index='bus_route_id_station_code', columns='date',
                                  values='morning_getin', aggfunc='sum').fillna(0)

kmeans = MiniBatchKMeans(n_clusters=200, random_state=1993)

%time kmeans.fit(df_cluster_pivot)

df_cluster_pivot['kmeans1'] = kmeans.predict(df_cluster_pivot)

combined = pd.merge(combined, df_cluster_pivot[['kmeans1']], left_on = 'bus_route_id_station_code', right_index=True, how='left')

Wall time: 555 ms


In [40]:
# Kmeans2
df_cluster = combined[['date','bus_route_id','station_code','noon_getin']].copy()
df_cluster['bus_route_id_station_code'] = df_cluster['bus_route_id'].astype(np.str)+'_'+df_cluster['station_code'].astype(np.str)
df_cluster_pivot = pd.pivot_table(data = df_cluster, index='bus_route_id_station_code', columns='date',
                                  values='noon_getin', aggfunc='sum').fillna(0)

kmeans = MiniBatchKMeans(n_clusters=200, random_state=1993)

%time kmeans.fit(df_cluster_pivot)

df_cluster_pivot['kmeans2'] = kmeans.predict(df_cluster_pivot)

combined = pd.merge(combined, df_cluster_pivot[['kmeans2']], left_on = 'bus_route_id_station_code', right_index=True, how='left')

Wall time: 965 ms


In [41]:
# Merge 
combined = pd.merge(combined, df_commuter, on = 'station_code',how='left')

In [42]:
# Afternoon-Getoff-Amount
df_afternoon_getoff_amount['getoff_date'] = pd.to_datetime(df_afternoon_getoff_amount['getoff_date'] )
combined = pd.merge(combined, df_afternoon_getoff_amount,
                 left_on = ['bus_route_id','date','station_code'],
                 right_on = ['bus_route_id','getoff_date','getoff_station_code'],
                 how='left')

combined = combined.drop(['getoff_date','getoff_station_code'], 1)
combined['afternoon_takeoff'] = combined['afternoon_takeoff'].fillna(0)

In [43]:
# 이전 버스와의 배차간격?
date_route_stataion_waittime['geton_date'] = pd.to_datetime(date_route_stataion_waittime['geton_date'] )
combined = pd.merge(combined, date_route_stataion_waittime, 
                                         left_on =['date','bus_route_id'] ,
                                         right_on =['geton_date','bus_route_id'],
                                             how='left')
combined = combined.drop(['geton_date'],1)

In [44]:
# 6~9, 9~12시 사이에 각기 다른 집단의 사람들이 몇명 탑승했는지
geton_bus_passender_cluster_count_morning['geton_date'] = pd.to_datetime(geton_bus_passender_cluster_count_morning['geton_date'] )
combined = pd.merge( combined, geton_bus_passender_cluster_count_morning , left_on = ['date','station_code'],
                                         right_on = ['geton_date', 'geton_station_code'],
                                         how = 'left')

combined = combined.drop(['geton_station_code','geton_date'],1)

In [45]:
# 6~9, 9~12시 사이에 각기 다른 집단의 사람들이 몇명 내렸는지
takeoff_bus_passender_cluster_count_noon['geton_date'] = pd.to_datetime(takeoff_bus_passender_cluster_count_noon['geton_date'] )
combined = pd.merge( combined, takeoff_bus_passender_cluster_count_noon , left_on = ['date','station_code'],
                                                                         right_on = ['geton_date', 'getoff_station_code'],
                                                                         how = 'left')

combined = combined.drop(['getoff_station_code','geton_date'],1)

In [46]:
# 기상데이터 -- 12시 전에 수집된 강수량
df_rain = pd.read_csv('../preprocessed_external_dataset/hourly_rain.csv')
df_rain['date'] = pd.to_datetime(df_rain['date'])

combined = pd.merge(combined, df_rain, on='date', how='left')

# 기상데이터 -- 전날 강수량
df_daily_rain = pd.read_csv('../preprocessed_external_dataset/daily_rain.csv')
df_daily_rain['date'] = pd.to_datetime(df_daily_rain['date'])
df_daily_rain.columns = ['prev_date','prev_daily_rain']

combined['prev_date'] = pd.to_datetime(combined['date']) - pd.Timedelta('1 day')
combined = pd.merge(combined, df_daily_rain, on='prev_date', how='left')

# 기상데이터 -- 12시 전에 수집된 운집량
df_cloud = pd.read_csv('../preprocessed_external_dataset/hourly_cloud.csv')
df_cloud['date'] = pd.to_datetime(df_rain['date'])

combined = pd.merge(combined, df_cloud, on='date', how='left')

In [47]:
# Google-Map을 통한 정류장의 주소정보
# 위경도 좌표를 입력값으로 해당 좌표의 주소를 크롤링한 pickle파일입니다.
geo_df2 = pd.read_pickle('../preprocessed_external_dataset/second_whole_dict.pickle')
combined['latlong_second'] = combined['latitude'].astype(np.str) +'_'+ combined['longitude'].astype(np.str)
combined['latlong_second'] = combined['latlong_second'].apply(lambda x: geo_df2.get(x))

In [49]:
# 주소 별 거주자 수
df_pop = pd.read_csv('../preprocessed_external_dataset/제주도_거주자수.csv')
combined['district'] = combined['latlong_second'].apply(lambda x: x.split(' ')[1].split(' ')[0])
combined = pd.merge(combined, df_pop, on='district', how='left')

In [50]:
combined = combined.rename(columns={"총_인구":"total_population", "남자_인구":"man_population", "여자_인구":"woman_population"})

In [51]:
# Station 정보
df_bus['time_diff'] = pd.to_datetime(df_bus['getoff_time'])-pd.to_datetime(df_bus['geton_time'])
df_bus['time_diff'] = df_bus['time_diff'].dt.seconds

# 각 정류장에서 평균 탑승 시간은 얼마인지
station_info = df_bus.groupby(['geton_station_code'])['time_diff'].mean().reset_index()
station_info.columns = ['station_code','avg_time_diff']

combined = pd.merge(combined, station_info, on ='station_code', how='left')

In [52]:
# 각 정류장에서 일평균 승차,하차 승객의 차이
station_passenger_in = df_bus.groupby(['geton_date','geton_station_code'])['user_card_id'].nunique().reset_index()
station_passenger_out = df_bus.groupby(['geton_date','getoff_station_code'])['user_card_id'].nunique().reset_index()

station_passenger_in.columns = ['date','station_code','passengers_in']
station_passenger_out.columns = ['date','station_code','passengers_out']

station_passenger_in = station_passenger_in.groupby('station_code')['passengers_in'].mean().reset_index()
station_passenger_out = station_passenger_out.groupby('station_code')['passengers_out'].mean().reset_index()

station_passenger = pd.merge(station_passenger_in, station_passenger_out, how='outer', on='station_code')

combined = pd.merge(combined, station_passenger, how='left', on='station_code')

In [53]:
combined['latitude_rank'] = combined['latitude'].rank()
combined['longitude_rank'] = combined['longitude'].rank()

In [54]:
drop_cols += ['prev_date', 'district']

In [55]:
# Drop unnecessary columns
drop_cols += ['id', 'date', 'station_name',] +\
            ['getin_user_count4_morning', 'takeoff_user_count4_noon', 'getin_user_count6_morning', 'takeoff_user_count6_noon',
            'prev_date','district',
            ]

In [56]:
train, test = combined[:n_trn][[col for col in combined.columns if col not in drop_cols]] , combined[n_trn:][[col for col in combined.columns if col not in drop_cols]]

In [57]:
# Label Encoding
cat_cols = [
    'latlong_second'
]

cat_cols += [
                'bus_route_id','station_code','dayofweek',
                'kmeans1','kmeans2',
                'bus_route_id_station_code','bus_route_id_station_code_weekend'
            ]

for col in tqdm_notebook(cat_cols):
    lbl = LabelEncoder()
    lbl.fit( train[col].tolist() + test[col].tolist() )
    train[col] = lbl.transform( train[[col]]  )
    test[col] = lbl.transform( test[[col]]  )
    
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [58]:
train.to_pickle('preprocessed_train.pickle')
test.to_pickle('preprocessed_test.pickle')